In [50]:
%run 2gis_functions.ipynb

Ready: 2023-03-31 03:39:00


In [60]:
findspark.init()
from pyspark.sql import SparkSession
try:
    spark.stop()
except:
    pass
spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_Tutorial')\
        .getOrCreate()
spark

# Новосибирск

In [143]:
name = 'novosib'
rubrics = pd.read_excel(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Backups\_Final\Мастер_таблица_2ГИС_Новосибирск_230322_2345.xlsx")[['Рубрика']].iloc[:70]

df = spark.read.parquet(f'df_filtered_{name}')
df.createOrReplaceTempView('df')

In [210]:
df_new = spark.sql('''

select 
rubricName
, time
, year(time) year_r
, hour(time) hour_r
, SUBSTR(time, 6, 5) date_r
, case when weekofyear(time) > 9 then 1 else weekofyear(time) end week_r
, case when extract(dayofweek from time) = 1 then 7
    else extract(dayofweek from time)-1
end week_day_r

from df
''')

df_new.write.mode('overwrite').parquet(f'df_dates_{name}')

In [344]:
df_new = spark.read.parquet(f'df_dates_{name}').where('year_r in (2020,2021,2022,2023) and month(time) < 3 ')
df_new.createOrReplaceTempView('df_new')

In [345]:
dff1 = count_dates('hour_r')
dff2 = count_dates('date_r')
dff3 = count_dates('week_r')
dff4 = count_dates('week_day_r')

In [346]:
t = ['Рубрика','Час','Количество запросов 2020','Количество запросов 2021','Количество запросов 2022','Количество запросов 2023',
    'max_2020','max_2021','max_2022','max_2023','min_2020','min_2021','min_2022','min_2023','up_2021','up_2022','up_2023','down_2021','down_2022','down_2023', 'delta_score']


In [347]:
t[1] = 'Час'
dff1.columns = t
t[1] = 'Дата'
dff2.columns = t
t[1] = 'Неделя'
dff3.columns = t
t[1] = 'День недели'
dff4.columns = t

In [348]:
with pd.ExcelWriter(f'df_dates_{name}.xlsx') as writer:
    dff1.to_excel(writer, sheet_name='Часы', index=False)
    dff2.to_excel(writer, sheet_name='Даты', index=False)
    dff3.to_excel(writer, sheet_name='Недели', index=False)
    dff4.to_excel(writer, sheet_name='Дни недели', index=False)

## Отрисовываем

In [ ]:
rubrics = pd.read_excel(r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Backups\_Final\Мастер_таблица_2ГИС_Новосибирск_230322_2345.xlsx")[['Рубрика']].iloc[:70]

make_linear_plots('files/novosib_dates.xlsx', rubrics, cnt=25)

In [25]:
copy_images_texts('files/novosib_dates.xlsx', r"C:\Users\Mamed\YandexDisk\_Documents\Проекты\2ГИС 2023\Новосибирск\Статья_Новосибирский распорядок_230401_1430.docx", True)

files/novosib_dates_report_230402_2213.docx


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:33<00:00,  2.97it/s]
